In [1]:
# Importation des librairies

import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import train_test_split
from lazypredict.Supervised import LazyClassifier
from sklearn.utils import all_estimators 
from sklearn.base import ClassifierMixin

# Ajuster les paramètres pour afficher toutes les lignes et colonnes

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

In [ ]:
# Importation des bibliothèques pour l'apprentissage automatique et la régression automatique

import autosklearn.regression as autosk_reg
import pandas as pd
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Définition des variables cibles pour l'analyse des temps d'intervention

cols_cible_type = ["PumpSecondsOnSite", "TurnoutTimeSeconds", "TravelTimeSeconds"]
cols_cible = [
    ["PumpSecondsOnSite_min", "PumpSecondsOnSite_mean", "PumpSecondsOnSite_max"],
    ["TurnoutTimeSeconds_min", "TurnoutTimeSeconds_mean", "TurnoutTimeSeconds_max"],
    ["TravelTimeSeconds_min", "TravelTimeSeconds_mean", "TravelTimeSeconds_max"]
    # NumPumpsAttending??
]

def load_df(col_cible_type):
    
    """
    Cette fonction charge un fichier CSV contenant un DataFrame et le retourne.

    Paramètres :

            col_cible_type (str): Chaîne de caractères utilisée pour construire le nom du fichier à charger. Le fichier attendu est nommé _df_ready_{col_cible_type}.csv et se trouve dans le répertoire ../data/.
    
    Retour :

           Le DataFrame chargé depuis le fichier CSV.

    """

    df = pd.read_csv(f"../data/_df_ready_{col_cible_type}.csv", sep=";", low_memory=False)
    # Contrôle
    # display(df.head(3))
    # display(df.info())
    return df

# Automatisation de la Modélisation et de l'Évaluation avec AutoML
# Ce code charge plusieurs ensembles de données, effectue un prétraitement en supprimant certaines colonnes cibles, applique AutoML avec AutoSklearn pour la régression
# et évalue les performances du modèle à l'aide des métriques MSE et R².

for index, name in enumerate(cols_cible_type):
    df = load_df(name)
    X = df.drop(["PumpSecondsOnSite_min", "PumpSecondsOnSite_mean", "PumpSecondsOnSite_max", "TurnoutTimeSeconds_min", "TurnoutTimeSeconds_mean",
                  "TurnoutTimeSeconds_max", "TravelTimeSeconds_min", "TravelTimeSeconds_mean", "TravelTimeSeconds_max", "NumPumpsAttending"], axis=1)
    display(X.head(2))
    print(X.shape)
    
    # Auto ML
    df_limited = df[df.CalYear >= 10]
    X = df_limited.drop(["PumpSecondsOnSite_min", "PumpSecondsOnSite_mean", "PumpSecondsOnSite_max", "TurnoutTimeSeconds_min", "TurnoutTimeSeconds_mean",
                  "TurnoutTimeSeconds_max", "TravelTimeSeconds_min", "TravelTimeSeconds_mean", "TravelTimeSeconds_max", "NumPumpsAttending"], axis=1)
    for col_cible in cols_cible[index]:
        print("//////////////////////////////////////////////////////////////////////////////////////////")
        print(col_cible)
        y = df_limited[col_cible]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

        # Créer et entraîner Auto-sklearn pour la régression
        automl = autosk_reg.AutoSklearnRegressor(
            time_left_for_this_task=80000,  # Temps total alloué pour l'entraînement (en secondes)
            per_run_time_limit=6000,  # Temps maximum pour chaque modèle testé
            n_jobs=-1,  # Utilise tous les cœurs disponibles
            ensemble_kwargs = {'ensemble_size': 10},  # Nombre maximal de modèles dans l'ensemble final
            seed=42,
        )

        # Entraînement
        automl.fit(X_train, y_train)
        # Prédiction
        y_pred = automl.predict(X_test)
        # Évaluation des performan ces
        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)

        print(f"Mean Squared Error: {mse}")
        print(f"R2 Score: {r2}")
        # Afficher un résumé détaillé des modèles et hyperparamètres
        print("\n********** Rapport détaillé des modèles **********\n")
        print(automl.show_models())
        print("\n********** Statistiques d'entraînement **********\n")
        print(automl.sprint_statistics())